In [7]:
import numpy as np
import cvxpy as cp
import random
import pandas as pd


In [8]:
n = 10000    # number of bidders
m = 10       # type of resources

A = pd.read_csv("A.csv")
pi = pd.read_csv("pi_2.csv")

A = np.array(A)
pi = np.array(pi)
pi = pi.T
b = np.array([1000 for _ in range(m)])  


In [9]:
def dynamicLearning(start, k, xp, xk):
	Anow = A[:, 0:k]
	Anow = Anow.T
	# print(Anow.shape)
	piNow = (pi.T)[0:k]
	piNow = np.reshape(piNow, (k,))
	bNow = b*(float(k/n))
	bNow = bNow.T

	temp = m+k
	y = cp.Variable(temp)
	e = np.ones(k)
	objective = cp.Minimize(bNow@(y[:m]) + e@(y[m:]))
	constraints = [(Anow@(y[:m])+ (y[m:]))>= piNow, y >= 0]
	prob = cp.Problem(objective, constraints)

	prob.solve()
	P = y[:m].value
	print(P)
	AT = A.T

	xpNow = xp
	xkNow = xk
	for i in range(start, 2*k):
		if i >= n:
			# print(xkNow)
			global xTrue
			xTrue = xkNow
			return xkNow
		
		cri = (AT[i].dot(P))
		if(pi[0][i] > cri):
			xpNow.append(1)
		else:
			xpNow.append(0)

		test = 1
		for j in range(m):
			temp1 = A[j,i]*xpNow[i]
			temp2 = b[j] - (A[j:j+1, :i]).dot(np.array(xkNow))
			# print(i)
			# print(len(xkNow))
			if temp1 > temp2:
				test = 0
				break
		if test == 1:
			xkNow.append(xpNow[i])
		else:
			xkNow.append(0)

	dynamicLearning(2*k, 2*k, xpNow, xkNow)


In [10]:
xk = [0]*50
xp = [0]*50
xTrue = []

tt = dynamicLearning(50, 50, xp, xk)

[ 9.91757979e-01  2.26331176e+00  3.01396748e-01  1.64481363e+00
  1.52408618e+00  7.54613422e-01 -2.20723176e-13  2.46111224e+00
 -2.22418841e-10  2.28117793e+00]
[2.35952874e+00 1.46123825e+00 2.82112213e-01 2.29398892e+00
 1.50929186e+00 1.31427353e+00 3.42334456e-11 2.62880478e+00
 1.81815872e-01 1.48171717e+00]
[1.07410566 2.10066455 0.57860929 1.48496951 1.2868929  1.25795536
 0.93724674 1.7951231  1.13328647 1.54773196]
[1.37781565 0.97572414 1.50104368 1.52341927 1.02266482 1.54797307
 0.84973505 2.13577168 0.73972631 1.52991067]
[1.34074516 1.24296322 1.16057998 1.13725442 1.43214709 1.36771119
 1.14892267 1.94757017 1.10533914 1.16822217]
[1.26973446 1.23877853 1.33114531 1.31699503 1.21777072 1.62161125
 1.38730174 1.48715167 1.16222857 1.21724671]
[1.26493434 1.37084484 1.31586146 1.25160164 1.30002242 1.43390233
 1.1865231  1.37060717 1.30698641 1.17893819]
[1.35048573 1.31842829 1.27062313 1.30281844 1.28386011 1.45560911
 1.27756702 1.14927728 1.22668259 1.25851709]


In [11]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)

prob.solve()

opv1 = prob.value

In [12]:
print(xTrue)
xTrue = np.array(xTrue)
opv2 = (pi.dot(xTrue))[0]
rateK50 = opv2/opv1

rateK50

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 

0.9781035169736172